In [1]:
import dataAck
import portfolio
allTickers = dataAck.getAllTickersPlain()
import params

In [2]:
print(str(portfolio.getToday()))

2017-08-15 00:00:00


In [3]:
from google.cloud import datastore, storage, logging
import pickle
import hashlib
def storeTrainingData(ticker, joinedData):
    storageClient = storage.Client('money-maker-1236')
    while True:
        try:
            bucket = storageClient.get_bucket(params.trainingDataCache)
            dayHash = hashlib.sha224((ticker + "_" +str(portfolio.getToday())).encode('utf-8')).hexdigest()
            blob = storage.Blob(dayHash, bucket)
            blob.upload_from_string(pickle.dumps(joinedData))
            print("STORING", dayHash)
            break
        except:
            print("UPLOAD BLOB ERROR:", str(sys.exc_info()))
            time.sleep(10)
    pass

def getTrainingData(ticker):
    storageClient = storage.Client('money-maker-1236')
    try:
        bucket = storageClient.get_bucket(params.trainingDataCache)
        dayHash = hashlib.sha224((ticker + "_" +str(portfolio.getToday())).encode('utf-8')).hexdigest()
        print("ATTEMPTING PULL", dayHash)
        blob = storage.Blob(dayHash, bucket)
        return pickle.loads(blob.download_as_string())
    except:
        return None
    pass


In [ ]:
while True:
    import random
    tickerToTrade = "SPY"#allTickers[random.randint(0, len(allTickers)) - 1]
    print(tickerToTrade)
    
    tData = getTrainingData(tickerToTrade)
    joinedData = None
    validTickers = None

    
    
    if tData is None:
        dataAck.logModel("Cache", {
            "type":"miss",
            "ticker":tickerToTrade,
            "day":str(portfolio.getToday())
        })

        tickersToPull = dataAck.getDataSourcesForTicker(tickerToTrade)
        print(tickersToPull)

        pulledData, validTickers = dataAck.downloadTickerData(tickersToPull)

        joinedData = dataAck.joinDatasets([pulledData[ticker] for ticker in pulledData])
        
        storeTrainingData(tickerToTrade, (joinedData, validTickers))
    else:
        joinedData = tData[0]
        validTickers = tData[1]
        dataAck.logModel("Cache", {
            "type":"hit",
            "ticker":tickerToTrade,
            "day":str(portfolio.getToday())
        })
        
    
    sManager = dataAck.seriesManager(validTickers)
    print(sManager.describe())

    import time
    import warnings
    import numpy as np
    warnings.filterwarnings("ignore")
    ##GET ALGOS INITIALLY GOOD
    runsSeen = 0
    while True:
        s = sManager.createSeries()
        while s.checkValidity(s.transformJoinedData(joinedData[:"2016-01-01"])) == False:
            s = sManager.createSeries()
            

        try:
            for defaultWindowSize in [5, 10, 22, 44]:
                for trees in [25, 50, 100, 150, 300]:
                    for predictionLength in [2, 3, 5, 7]:
                        if random.uniform(0,1) < 0.5: ##RANDOMLY SKIP
                            continue
                        b = dataAck.algoBlob(s, defaultWindowSize, trees, predictionLength, tickerToTrade)
                        algoReturn, factorReturn, predictions =  b.makePredictions(portfolio.prepareDataForModel(b, joinedData), daysToCheck = None, earlyStop = True)
                        if algoReturn is None:
                            toLog = factorReturn
                            toLog["modelDescription"] = str(b.describe())
                            dataAck.logModel("Model Stopped Early", toLog)
                            print("Model Stopped Early", toLog)
                            continue
                        metrics = dataAck.vizResults(algoReturn[:-252], factorReturn[:-252], False)
                        print("TRAIN:", metrics)
                        if np.isnan(metrics["SHARPE"]) == True:
                            raise ValueError('SHARPE IS NAN SO FAULTY SERIES')
                        if metrics["SHARPE"] > 1.0 and metrics["ACTIVITY"] > 0.7 and metrics["BETA"] < 0.6:
                            ##STORE
                            testMetrics = dataAck.vizResults(algoReturn[-252:], factorReturn[-252:], False)
                            print("TEST:", testMetrics)
                            print("TODAY:", b.makeTodayPrediction(portfolio.prepareDataForModel(b, joinedData)))

                            dataAck.storeModel(b, metrics, testMetrics)
                        else:
                            dataAck.logModel("Model Skipped", {
                                "modelDescription":str(b.describe()),
                                "sharpe":metrics["SHARPE"], ##OVERLOADED IN FAIL
                                "activity":metrics["ACTIVITY"],
                                "beta":metrics["BETA"]
                            })
        except:
            print("FAILED", s.describe())
            dataAck.logModel("Series Failed", {
                "seriesDescription":str(s.describe())
            })
            time.sleep(10)

        runsSeen += 1

        if runsSeen > 10:
            ##START NEW TICKER
            dataAck.logModel("Search Update", {
                "message":"restarting search with different ticker",
                "currentTicker":tickerToTrade
            })
            break

        

SPY
ATTEMPTING PULL 65f5a82ffb072b2c7672d268a46976ec3bd87f86c3b36568b2284f83
['AGG', 'DXJ', 'EWC', 'EWJ', 'FXE', 'GDX', 'GLD', 'IAU', 'IEF', 'LQD', 'SHY', 'SLV', 'TIP', 'TLT', 'USO', 'VIG', 'XLP', 'XLU', 'XLV', 'SPY']
(5, ('DXJ', 10, 12, None, 2), 2, 'SPY')
SERIES (5, ('DXJ', 10, 12, None, 2), 2, 'SPY') WINDOW 5 TREES 25
CONTINUING SHARPE: 0.710085414656 BETA: -0.0673270244968
Model Stopped Early {'sharpe': -0.0029067008746106256, 'beta': 0.029638310287128867, 'activity': 0.585, 'period': 'first 600 days', 'modelDescription': "((5, ('DXJ', 10, 12, None, 2), 2, 'SPY'), 5, 25)"}
(5, ('DXJ', 10, 12, None, 2), 3, 'SPY')
SERIES (5, ('DXJ', 10, 12, None, 2), 3, 'SPY') WINDOW 5 TREES 25


Process ForkProcess-80:
Process ForkProcess-77:
Process ForkProcess-87:
Process ForkProcess-79:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/prohb125/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/prohb125/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/prohb125/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/prohb125/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/prohb125/anaconda3/lib/python3.6/warnings.py", line 164, in _add_filter
    filters.remove(item)
  File "/home/prohb125/walkforwardTrader/dataAck.py", line 309, in runDayChunking
    pred = self.runDay(xVals[:int(i)], yVals[:int(i)], xVals[int(

FAILED ('DXJ', 10, 12, None, 2)


KeyboardInterrupt: 

ERROR:tornado.general:Uncaught exception, closing connection.
Traceback (most recent call last):
  File "/home/prohb125/anaconda3/lib/python3.6/site-packages/zmq/eventloop/zmqstream.py", line 414, in _run_callback
    callback(*args, **kwargs)
  File "/home/prohb125/anaconda3/lib/python3.6/site-packages/tornado/stack_context.py", line 275, in null_wrapper
    return fn(*args, **kwargs)
  File "/home/prohb125/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 276, in dispatcher
    return self.dispatch_shell(stream, msg)
  File "/home/prohb125/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 228, in dispatch_shell
    handler(stream, idents, msg)
  File "/home/prohb125/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 412, in execute_request
    self._abort_queues()
  File "/home/prohb125/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 628, in _abort_queues
    self._abort_queue(stream)
  File "/home/prohb125/